In [1]:
import typing as T  # pylint: disable=unused-import

import numpy as np
import numpy.typing as npt

from pydrake.solvers import (  # pylint: disable=import-error, no-name-in-module, unused-import
    MakeSemidefiniteRelaxation,
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    IpoptSolver,
    SnoptSolver,
    GurobiSolver,
    MosekSolver,
    MosekSolverDetails,
    SolverOptions,
    CommonSolverOption,
)

from pydrake.geometry.optimization import (  # pylint: disable=import-error, no-name-in-module
    HPolyhedron,
    Point,
    ConvexSet,
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
)

from pydrake.symbolic import ( # pylint: disable=import-error, no-name-in-module, unused-import
    Polynomial,
    Variable,
    Variables,
    Expression,
)  

from contact_cart import ContactCart

import plotly.graph_objects as go  # pylint: disable=import-error
from plotly.express.colors import sample_colorscale  # pylint: disable=import-error
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
def make_plots(qcqp_scaling_factor):
    cart = ContactCart(v0=-7, qcqp_scaling_factor = qcqp_scaling_factor, verbose = True, Qx =1, Rf=0.01, umax=50)

    x_trajs = []
    v_trajs = []
    f_trajs = []
    u_trajs = []
    lcp_trajs = []
    cost_trajs = []

    colors = {"micp": "red", "sdp":"blue", "dec qcqp":"green", "dec qcqp warmstart": "yellow", "chordal sdp": "purple"}

    # functions = [(cart.solve_nonconvex_qcqp_sdp_relaxation(),"sdp"), (cart.solve_micp_with_gcs(), "micp"), (cart.solve_qcqp_convex_diff_decomposition(), "dec qcqp"), (cart.solve_sdp_convex_diff_decomposition(), "dec sdp")]
    # functions = [(cart.solve_nonconvex_qcqp_sdp_relaxation(),"sdp"), (cart.solve_micp_with_gcs(), "micp"), (cart.solve_our_qcqp_convex_diff_decomposition(), "dec qcqp")]
    functions = [(cart.solve_nonconvex_qcqp_sdp_relaxation(),"sdp"), 
                 (cart.solve_micp_with_gcs(), "micp"), 
                 (cart.solve_our_qcqp_convex_diff_decomposition(), "dec qcqp"),
                #  (cart.solve_bad_qcqp_convex_diff_decomposition(), "bad qcqp"),
                 (cart.solve_chordal_nonconvex_qcqp_sdp_relaxation(), "chordal sdp"),
                 (cart.solve_our_qcqp_convex_diff_decomposition_with_warmstart(), "dec qcqp warmstart"),
                 ]

    for (stuff, name) in functions:
        cost, x_traj, v_traj, f_traj, u_traj, lcp_v_traj = stuff
        x_trajs.append((x_traj, name))
        v_trajs.append((v_traj, name))
        f_trajs.append((f_traj, name))
        u_trajs.append((u_traj, name))
        lcp_trajs.append((lcp_v_traj, name))
        cost_trajs.append((np.ones(len(x_traj))*cost, name))

    fig = go.Figure()

    time = np.array(list(range(cart.N)))

    # Create subplots
    fig = make_subplots(rows=6, cols=1)


    for (x_traj, name) in x_trajs:
        fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name]) ), row=1, col=1)

    for (x_traj, name) in v_trajs:
        fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=2, col=1)    

    for (x_traj, name) in f_trajs:
        fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=3, col=1)    

    for (x_traj, name) in u_trajs:
        fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=4, col=1)    

    for (x_traj, name) in lcp_trajs:
        fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=5, col=1)    

    for (x_traj, name) in cost_trajs:
        fig.add_trace(go.Scatter(x=time, y=x_traj, mode='lines', name=name, line=dict(color=colors[name])), row=6, col=1)    


    # Update layout
    fig.update_layout(height=1200, width=800, title_text="Solver Comparisons")

    fig.update_yaxes(title_text="Position", row=1, col=1)
    fig.update_yaxes(title_text="Velocity", row=2, col=1)
    fig.update_yaxes(title_text="Normal Force", row=3, col=1)
    fig.update_yaxes(title_text="Control", row=4, col=1)
    fig.update_yaxes(title_text="LCP violation", row=5, col=1)
    fig.update_yaxes(title_text="Cost", row=6, col=1)

    # Show plot
    fig.show()

In [3]:
make_plots(0.1)

INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.
INFO:drake:Finished 100 rounding trials.


nonconvex QCQP relaxaion with MOSEK took 0.266s
solve successful!
33.64993910594622
SolutionResult.kSolutionFound
Solver is Mosek
time 0.25592780113220215
rescode 0
solution_status 1
-------------------
Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-01
GCS MICP took 0.224s
solve successful!
33.64993921817789
SolutionResult.kSolutionFound
Solver is Gurobi
-------------------
our convex decomposition QCQP took 0.006s
solve successful!
27.168949325959325
SolutionResult.kSolutionFound
Solver is Mosek
time 0.0011148452758789062
rescode 0
solution_status 1
27.168949325959325
-------------------
chordal noconvex-qcqp SDP relaxation took 0.052s
solve successful!
33.64995931467382
SolutionResult.kSolutionFound
Solver is Mosek
time 0.01025390625
rescode 0
solution_status 1
-------------------
our convex decomposition QCQP took 0.006s
solve successful!
27.168949325959325
SolutionResult.kSolutionFound
Solver is Mosek
time 0.001074075698852539
rescode 0
solu

In [5]:
def run_a_test_for_suboptimality_gap(Qx=1,Rf=0.01):
    sdp_violations = []
    qcqp_violations = []
    for v0 in range(-15, 15):
        cart = ContactCart(v0=v0, qcqp_scaling_factor = 0.01, verbose = False, Qx =Qx, Rf=Rf, umax=50)

        chordal_sdp_cost, _, _, _, _, _ = cart.solve_chordal_nonconvex_qcqp_sdp_relaxation()
        micp_cost, _, _, _, _, _ = cart.solve_micp_with_gcs()
        qcqp_ipopt_cost, _, _, _, _, _ = cart.solve_our_qcqp_convex_diff_decomposition_with_warmstart()
        sdp_violations.append(chordal_sdp_cost/micp_cost)
        qcqp_violations.append(qcqp_ipopt_cost/micp_cost)

    print("SDP", np.min(sdp_violations), np.max(sdp_violations))
    print("QCQP IPOPT", np.min(qcqp_violations), np.max(qcqp_violations))
    print(qcqp_violations)

run_a_test_for_suboptimality_gap(0.001)

INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.
INFO:drake:Finished 100 rounding trials.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.
INFO:drake:Finished 100 rounding trials.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.
INFO:drake:Finished 100 rounding trials.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Mosek with convex_relaxation=true and preprocessing=false and rounding.
INFO:drake:Finished 100 rounding trials.
INFO:drake:Solved GCS s

SDP 0.9375290100800178 1.000491728983575
QCQP IPOPT 0.9999975290939488 1.146434622407941
[1.0000012216252459, 0.9999996470015032, 0.999999887915264, 1.0000000256295956, 0.9999989161333601, 0.9999993183169378, 0.9999975290939488, 1.0000018106500839, 1.146434622407941, 0.9999999363647044, 1.0000001168629653, 1.0000015960300739, 1.027715661093529, 0.9999984367241406, 0.9999997327808093, 1.0004756365275784, 0.9999983827115138, 0.9999982895503531, 0.9999998669590758, 0.9999999418463766, 0.9999997757936923, 0.9999999613450412, 0.9999999022404111, 0.9999999107239123, 0.9999998935822574, 0.9999999522195178, 0.9999997689771133, 0.9999999008746869, 0.9999999475209926, 0.9999999950502386]


In [22]:
def run_a_test_for_relaxation_gap():
    sdp_violations = []
    qcqp_violations = []
    for v0 in range(-15, 15):
        cart = ContactCart(v0=v0, qcqp_scaling_factor = 0.01, verbose = False,umax=60)

        x_trajs = []
        v_trajs = []
        f_trajs = []
        u_trajs = []
        lcp_trajs = []
        cost_trajs = []

        colors = {"micp": "red", "sdp":"blue", "qcqp":"green", "ipopt" : "yellow"}

        sdp_cost, _, _, _, _, _ = cart.solve_nonconvex_qcqp_sdp_relaxation()
        micp_cost, _, _, _, _, _ = cart.solve_micp_with_gcs()
        qcqp_cost, _, _, _, _, _ = cart.solve_qcqp_convex_diff_decomposition()
        sdp_violations.append(sdp_cost/micp_cost)
        qcqp_violations.append(qcqp_cost/micp_cost)

    print(sdp_violations)
    print(qcqp_violations)

run_a_test_for_relaxation_gap()

INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi with convex_relaxation=false and preprocessing=false and no rounding.
INFO:drake:Solved GCS shortest path using Gurobi

[0.5068239586429778, 0.6135436680344605, 0.7363168410681461, 0.8615047199932495, 0.9610163364920047, 1.0000029016673078, 1.0000010377943793, 0.9999999562211773, 0.9999999270282791, 1.0000000510387386, 0.999999872353953, 1.0000001394975055, 1.000000193248796, 0.9999998013613419, 1.000003282502361, 1.0000001634685214, 1.00000024169303, 1.000013463890352, 0.999998175248415, 0.9999998157731375, 1.000000053263569, 1.000000626195406, 1.0000001666886862, 1.0000001745222131, 0.9999996674530701, 1.000001331859278, 1.0000006629701725, 0.9999999993608861, 1.0000000214951215, 0.9999999984458244]
[1.2760756320977045, 1.404298542406176, 1.528804749881515, 1.5964030663188307, 1.5760289970785242, 1.4378695009136147, 1.1982000912719815, 0.9614953433113059, 0.826732018842995, 0.6990966423357222, 0.6797235598403361, 0.7057165564016639, 0.7403210082344631, 0.7864153739760515, 0.8480603137994767, 0.9218835796919465, 0.9869413518579323, 1.0034561308323813, 1.0033150993389344, 1.0026436281445228, 1.001969284

In [11]:
np.linalg.eigvals(np.array([[1,1],[1,1]]))

array([2., 0.])

In [19]:
def extract_moments_from_spectrahedron_prog_variables(vector:npt.NDArray, dim:int)-> T.Tuple[float, npt.NDArray, npt.NDArray]:
    # spectrahedron program variables stores vectors in the following array: m1, m2[triu], m0
    # indecis
    ind1 = dim
    ind2 = ind1 + int(dim*(dim+1)/2)
    # get moments
    m1 = vector[:ind1]
    m2 = np.zeros((dim, dim), dtype=vector.dtype)
    triu_ind = np.triu_indices(dim)
    m2[triu_ind[0], triu_ind[1]] = vector[ind1: ind2]
    m2[triu_ind[1], triu_ind[0]] = vector[ind1: ind2]
    m0 = vector[ind2]
    return m0, m1, m2


# generate a vertex for n -> n+1
prog = MathematicalProgram()

x1 = prog.NewContinuousVariables(1)[0]
v1 = prog.NewContinuousVariables(1)[0]
f1 = prog.NewContinuousVariables(1)[0]
u1 = prog.NewContinuousVariables(1)[0]
x2 = prog.NewContinuousVariables(1)[0]
v2 = prog.NewContinuousVariables(1)[0]

prog.AddLinearConstraint(x2 >= 0)
prog.AddLinearConstraint(f1 >= 0)
# prog.AddLinearConstraint(x2 == x1 + self.h * v2)
# prog.AddLinearConstraint(v2 == v1 + self.h/self.m * (u1+f1) )
prog.AddConstraint(x2*f1 == 0)
# prog.AddLinearConstraint(u1<= self.umax)
sdp_prog = MakeSemidefiniteRelaxation(prog)
_, _, m2 = extract_moments_from_spectrahedron_prog_variables(sdp_prog.decision_variables(), 6)
m2.shape

(6, 6)